<a href="https://colab.research.google.com/github/MuhammadBilall/FIX-Parser/blob/main/FIX_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
dataFolder = '/content/drive/MyDrive/FIX-Messages/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import os
import glob
dataFiles = []
txt_files = glob.glob(dataFolder + "*.txt")
print(txt_files)
for filename in txt_files:  
    print("Reading File: ", filename)
    data = pd.read_fwf(filename, header=None)
    dataFiles.append(data)

#print(dataFiles[0])

['/content/drive/MyDrive/FIX-Messages/2020-08-12 Messages.txt', '/content/drive/MyDrive/FIX-Messages/2020-08-11 Messages.txt']
Reading File:  /content/drive/MyDrive/FIX-Messages/2020-08-12 Messages.txt
Reading File:  /content/drive/MyDrive/FIX-Messages/2020-08-11 Messages.txt


In [3]:
def GetTagValue(msg, tag):
    splits = msg.split("\001")
    for s in splits:
        temp = s.split("=")
        if temp[0] == tag:
            return temp[1]
    return -1    

In [6]:
def ToDataFrame(dataFiles, tags):
    dataChunks = []

    for data in dataFiles:
        rows = []

        for i in range(len(data)):
            row = []

            for tag in tags:
                msg = data.at[i,0]
                value = GetTagValue(msg, tag)
                #print("msg num: ", i, "tag: ", tag, "value: ", value)
                row.append(value)

            rows.append(row)

        dataChunks.append(pd.DataFrame(rows, columns = tags))
        
    dataFrame = pd.concat(dataChunks, ignore_index=True)
    return dataFrame

In [7]:
tags = ['49', '56', '52']
dataFrame = ToDataFrame(dataFiles, tags)
print(dataFrame)

            49            56                     52
0      WEXDROP  VELOCITYDROP  20200812-13:33:58.389
1      WEXDROP  VELOCITYDROP  20200812-13:33:58.391
2      WEXDROP  VELOCITYDROP  20200812-13:33:58.396
3      WEXDROP  VELOCITYDROP  20200812-13:33:58.398
4      WEXDROP  VELOCITYDROP  20200812-13:33:58.405
...        ...           ...                    ...
64233  WEXDROP  VELOCITYDROP  20200811-19:59:30.068
64234  WEXDROP  VELOCITYDROP  20200811-19:59:27.006
64235  WEXDROP  VELOCITYDROP  20200811-19:59:27.007
64236  WEXDROP  VELOCITYDROP  20200811-19:59:30.051
64237  WEXDROP  VELOCITYDROP  20200811-19:59:30.052

[64238 rows x 3 columns]


In [5]:
dataFrame.to_csv(os.path.join(dataFolder, 'CleanData.csv'), index=False)